In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from tfidf_featurizer import combine_features, tfidf_featurize
from training import train, do_ensemble, do_train
from training_traditional import train_and_evaluate
from eval_erisk import evaluate, ensemble_vote
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow
import numpy as np

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


### With max_size = 10 and new data, sample_weights=10, oversample False, include_new_data=True

In [3]:
y_preds = {}

In [4]:
# prepare training data

In [5]:
train_users, y_train, test_users, y_test, train_samples, X_train, X_test = windowfy(window_size=10, max_size=20, sample_weights_size=20, is_oversample=False, include_new_data=True)
feats_train, feats_test = featurize(calculate_feats=True, include_feats=["first_prons","sentiment","nssi"], 
                       train_users=train_users, test_users=test_users, discretize=True, scale=True)
tfidf_train, tfidf_test = tfidf_featurize(train_users, test_users, max_features=50000)

feats_train_comb, feats_test_comb = combine_features([tfidf_train, feats_train], [tfidf_test, feats_test])

feats_train_comb = feats_train_comb.toarray()
feats_test_comb = feats_test_comb.toarray()

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%
Oversampling train users
After oversample: positive messages: 2792, negative messages: 3491
Data size: 6283

Finished windowfying
Featurizing calculate_feats=True, normalize=False, discretize=True, discretize_size=10, include_feats=['first_prons', 'sentiment', 'nssi']
Initialized numpy random and tensorflow random seed at 42
Data size: 6283, 6283
Data size: 4650, 4650
Calculating first prons
Calculating sentiment
Calculating NSSI words
Calculating first prons
Calculating sentiment
Calculating NSSI words
Scaling features
Training features shape: (6283, 8)
Test features shape: (4650, 8)
Discretizing


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:188: UserWarning: Feature 5 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:209: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  centers = km.fit(column[:, None]).cluster_centers_[:, 0]
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 6 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


Is the combined the same from tfidf: False


In [6]:
# traditional classifiers

In [7]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="svm", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["svm"] = y_pred

              precision    recall  f1-score   support

           0       0.82      0.98      0.90      3602
           1       0.82      0.27      0.40      1048

    accuracy                           0.82      4650
   macro avg       0.82      0.62      0.65      4650
weighted avg       0.82      0.82      0.78      4650

[[3541   61]
 [ 769  279]]
{'precision': 0.7313432835820896, 'recall': 0.47115384615384615, 'F1': 0.5730994152046783, 'ERDE_5': 0.25612439101077644, 'ERDE_50': 0.14048589105175943, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5507598630163498}


In [8]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="bayes", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["bayes"] = y_pred

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      3602
           1       0.56      0.42      0.48      1048

    accuracy                           0.79      4650
   macro avg       0.70      0.66      0.68      4650
weighted avg       0.78      0.79      0.78      4650

[[3254  348]
 [ 606  442]]
{'precision': 0.5338983050847458, 'recall': 0.6057692307692307, 'F1': 0.5675675675675675, 'ERDE_5': 0.27753497377746994, 'ERDE_50': 0.12889470125021749, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5454436481223696}


In [9]:
# deep learning classifiers
batch_size=32
model_name = "cnn_model"
iterations = 10

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=32
Generating embeddings
Data size: 6283
Training with callback
Restoring model weights from the end of the best epoch.
Epoch 00042: early stopping
Evaluating
146/146 [==============================] - 3s 12ms/step - loss: 0.4265 - tp: 366.0000 - fp: 121.0000 - tn: 3481.0000 - fn: 682.0000 - accuracy: 0.8273 - precision: 0.7515 - recall: 0.3492 - f1_metric: 0.2493
Test Score: 0.4265153408050537
Test Accuracy: 366.0
146/146 [==============================] - 2s 9ms/step
Entered here
              precision    recall  f1-score   support

           0       0.84      0.97      0.90      3602
           1       0.75      0.35      0.48      1048

    accuracy                           0.83      4650
   macro avg       0.79      0.66      0.69      4650
weighted avg       0.82      0.83      0.80      4650

[[3481  121]
 [ 682  366]]
Finished training and evaluation
{'precision': 0.7108433734939759, 'recall': 0.567307

In [ ]:
# deep learning classifiers
batch_size=32
model_name = "lstm_model_32"
iterations = 1

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

if iterations > 0:
    y_preds[model_name] = model_resuls[max(model_resuls.keys())]

In [ ]:
print(y_preds["cnn_model"].flatten())

In [ ]:
# ensemble

In [ ]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["cnn_model"].flatten(), y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

In [ ]:
y_pred = ensemble_vote(np.array([y_preds["bayes"], y_preds["cnn_model"].flatten(), y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

In [ ]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["bayes"], y_preds["lstm_model_32"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

In [ ]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["bayes"], y_preds["cnn_model"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)